# Data related dev

## Dataset

In [1]:
import sys
sys.path.append("../../")

In [2]:
from ptgnn.dataset import RSDataset, BindingAffinityDataset, BaceDataset, Tox21Dataset, OGBDataset

In [3]:
ogb_train = OGBDataset(split="train", ds_name="hiv")
ogb_test = OGBDataset(split="test", ds_name="hiv")
ogb_val = OGBDataset(split="val", ds_name="hiv")
print(ogb_train)
print(ogb_test)
print(ogb_val)

OGBDataset(32274)
OGBDataset(4074)
OGBDataset(3994)


In [4]:
bace_train = BaceDataset(split='train')
bace_test = BaceDataset(split='test')
bace_val = BaceDataset(split='val')
print(bace_train)
print(bace_test)
print(bace_val)

BaceDataset(1060)
BaceDataset(302)
BaceDataset(151)


In [5]:
tox_train = Tox21Dataset(split='train')
tox_test = Tox21Dataset(split='test')
tox_val = Tox21Dataset(split='val')
print(tox_train)
print(tox_test)
print(tox_val)

Tox21Dataset(5413)
Tox21Dataset(1544)
Tox21Dataset(777)


In [3]:
rs_train = RSDataset(split='train')
rs_test = RSDataset(split='test')
rs_val = RSDataset(split='val')
print(rs_train)
print(rs_test)
print(rs_val)

Processing...
55084it [06:13, 147.59it/s]
Done!
Processing...
11680it [01:21, 142.81it/s]
Done!
Processing...
11748it [01:24, 139.38it/s]
Done!


RSDataset(55068)
RSDataset(11676)
RSDataset(11740)


In [4]:
bindingaffinity_train = BindingAffinityDataset(split="train")
bindingaffinity_test = BindingAffinityDataset(split="test")
bindingaffinity_val = BindingAffinityDataset(split="val")
print(bindingaffinity_train)
print(bindingaffinity_test)
print(bindingaffinity_val)

Processing...
48384it [02:50, 283.26it/s]
Done!
Processing...
10368it [00:39, 262.47it/s]
Done!
Processing...
10368it [00:40, 254.84it/s]
Done!


BindingAffinityDataset(48344)
BindingAffinityDataset(10358)
BindingAffinityDataset(10354)


## Loader

In [2]:
from ptgnn.loading.load import UniversalLoader
from ptgnn.dataset import BindingAffinityDataset, BaceDataset

In [3]:
ba1 = BindingAffinityDataset(split="val")
loader = UniversalLoader(dataset=ba1, n_neighbors_in_circle=3, batch_size=3)

In [3]:
for batch in loader:
    print(batch)
    break

DataBatch(x=[130, 118], edge_index=[2, 378], edge_attr=[378, 80], pos=[130, 6], parallel_node_index=[130], y=[3], batch=[130], ptr=[4], circle_index=[130, 5])


In [4]:
batch.batch

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [5]:
batch.circle_index

tensor([[ -1,  -1,  -1,  -1,  -1],
        [  3,   7,   5,   3,   7],
        [  0,   5,   7,   0,   5],
        [  9,  11,  13,   9,  11],
        [  0,   7,   3,   0,   7],
        [ 38,  45,  43,  38,  45],
        [  0,   3,   5,   0,   3],
        [ -1,  -1,  -1,  -1,  -1],
        [  2,  13,  11,   2,  13],
        [ 15,  19,  17,  15,  19],
        [  2,   9,  13,   2,   9],
        [ -1,  -1,  -1,  -1,  -1],
        [  2,  11,   9,   2,  11],
        [ -1,  -1,  -1,  -1,  -1],
        [  8,  17,  19,   8,  17],
        [ 21,  25,  23,  21,  25],
        [  8,  19,  15,   8,  19],
        [ -1,  -1,  -1,  -1,  -1],
        [  8,  15,  17,   8,  15],
        [ -1,  -1,  -1,  -1,  -1],
        [ 14,  23,  25,  14,  23],
        [ 27,  29,  31,  27,  29],
        [ 14,  25,  21,  14,  25],
        [ -1,  -1,  -1,  -1,  -1],
        [ 14,  21,  23,  14,  21],
        [ -1,  -1,  -1,  -1,  -1],
        [ 20,  31,  29,  20,  31],
        [ 33,  37,  35,  33,  37],
        [ 20,  27,  

In [6]:
batch[0]

Data(x=[46, 118], edge_index=[2, 136], edge_attr=[136, 80], pos=[46, 6], parallel_node_index=[46], y=[1])

In [7]:
ba1[0].circle_index

[[],
 [3, 7, 5],
 [0, 5, 7],
 [9, 11, 13],
 [0, 7, 3],
 [38, 45, 43],
 [0, 3, 5],
 [],
 [2, 13, 11],
 [15, 19, 17],
 [2, 9, 13],
 [],
 [2, 11, 9],
 [],
 [8, 17, 19],
 [21, 25, 23],
 [8, 19, 15],
 [],
 [8, 15, 17],
 [],
 [14, 23, 25],
 [27, 29, 31],
 [14, 25, 21],
 [],
 [14, 21, 23],
 [],
 [20, 31, 29],
 [33, 37, 35],
 [20, 27, 31],
 [],
 [20, 29, 27],
 [],
 [26, 35, 37],
 [39, 41],
 [26, 37, 33],
 [],
 [26, 33, 35],
 [],
 [32, 41],
 [4, 43, 45],
 [32, 39],
 [],
 [4, 45, 38],
 [],
 [4, 38, 43],
 []]

In [49]:
type(ba1[0].circle_index)

list

In [4]:
from ptgnn.loading.custom_assembly import custom_loader

In [5]:
loader = custom_loader(ba1, n_neighbors_in_circle=3, sampler="single_conformer_sampler", batch_size=3, dataframe=ba1.dataframe)

In [6]:
for batch in loader:
    print(batch)
    break

DataBatch(x=[428, 118], edge_index=[2, 1252], edge_attr=[1252, 80], pos=[428, 6], parallel_node_index=[428], y=[6], batch=[428], ptr=[7], circle_index=[428, 5])


In [19]:
import numpy as np
print(np.unique(batch.batch.numpy(), return_counts=True))

(array([0, 1, 2, 3, 4, 5], dtype=int64), array([66, 66, 68, 68, 80, 80], dtype=int64))


In [10]:
# test for ptree transformation
bac1 = BaceDataset(split='val', graph_mode="edge", transformation_mode="chienn_tree_basic")

In [34]:
bac1 = BaceDataset(split='test', graph_mode="edge", transformation_mode="chienn_tree_basic")

Processing...
1060it [00:24, 43.07it/s] 
151it [00:07, 19.06it/s]
302it [00:09, 32.24it/s] 
Done!


In [11]:
bac1[0]

Data(x=[202, 118], edge_index=[2, 578], edge_attr=[578, 80], pos=[202, 6], ptree=[202], y=[1])

In [12]:
loader = UniversalLoader(dataset=bac1, n_neighbors_in_circle=3, batch_size=3)

In [32]:
for batch in loader:
    print(batch)
    break

DataBatch(x=[576, 118], edge_index=[2, 1642], edge_attr=[1642, 80], pos=[576, 6], y=[3], batch=[576], ptr=[4], ptree=[576])


In [5]:
bac1[0].ptree

['{"P": [1, {"Z": [3, 7, 5]}]}',
 '{"P": [0, {"Z": [9, 13, 11]}]}',
 '{"P": [3, {"Z": [1, 5, 7]}]}',
 '{"P": [2]}',
 '{"P": [5, {"Z": [1, 7, 3]}]}',
 '{"P": [4]}',
 '{"P": [7, {"Z": [1, 3, 5]}]}',
 '{"P": [6]}',
 '{"P": [9, {"Z": [0, 11, 13]}]}',
 '{"P": [8, {"Z": [15, 19, 17]}]}',
 '{"P": [11, {"Z": [0, 13, 9]}]}',
 '{"P": [10]}',
 '{"P": [13, {"Z": [0, 9, 11]}]}',
 '{"P": [12]}',
 '{"P": [15, {"Z": [8, 17, 19]}]}',
 '{"P": [14, {"Z": [21]}]}',
 '{"P": [17, {"Z": [8, 19, 15]}]}',
 '{"P": [16]}',
 '{"P": [19, {"Z": [8, 15, 17]}]}',
 '{"P": [18]}',
 '{"P": [21, {"Z": [14]}]}',
 '{"P": [20, {"Z": [23, 25, 27]}]}',
 '{"P": [23, {"Z": [20, 27, 25]}]}',
 '{"P": [22, {"Z": [29, 33, 31]}]}',
 '{"P": [25, {"Z": [20, 23, 27]}]}',
 '{"P": [24, {"Z": [42, 201, 199]}]}',
 '{"P": [27, {"Z": [20, 25, 23]}]}',
 '{"P": [26]}',
 '{"P": [29, {"Z": [22, 31, 33]}]}',
 '{"P": [28, {"Z": [35, 37, 39]}]}',
 '{"P": [31, {"Z": [22, 33, 29]}]}',
 '{"P": [30]}',
 '{"P": [33, {"Z": [22, 29, 31]}]}',
 '{"P": [32]}

In [14]:
batch.ptree

['{"P": [1, {"Z": [3, 7, 5]}]}',
 '{"P": [0, {"Z": [9, 13, 11]}]}',
 '{"P": [3, {"Z": [1, 5, 7]}]}',
 '{"P": [2]}',
 '{"P": [5, {"Z": [1, 7, 3]}]}',
 '{"P": [4]}',
 '{"P": [7, {"Z": [1, 3, 5]}]}',
 '{"P": [6]}',
 '{"P": [9, {"Z": [0, 11, 13]}]}',
 '{"P": [8, {"Z": [15, 19, 17]}]}',
 '{"P": [11, {"Z": [0, 13, 9]}]}',
 '{"P": [10]}',
 '{"P": [13, {"Z": [0, 9, 11]}]}',
 '{"P": [12]}',
 '{"P": [15, {"Z": [8, 17, 19]}]}',
 '{"P": [14, {"Z": [21]}]}',
 '{"P": [17, {"Z": [8, 19, 15]}]}',
 '{"P": [16]}',
 '{"P": [19, {"Z": [8, 15, 17]}]}',
 '{"P": [18]}',
 '{"P": [21, {"Z": [14]}]}',
 '{"P": [20, {"Z": [23, 25, 27]}]}',
 '{"P": [23, {"Z": [20, 27, 25]}]}',
 '{"P": [22, {"Z": [29, 33, 31]}]}',
 '{"P": [25, {"Z": [20, 23, 27]}]}',
 '{"P": [24, {"Z": [42, 201, 199]}]}',
 '{"P": [27, {"Z": [20, 25, 23]}]}',
 '{"P": [26]}',
 '{"P": [29, {"Z": [22, 31, 33]}]}',
 '{"P": [28, {"Z": [35, 37, 39]}]}',
 '{"P": [31, {"Z": [22, 33, 29]}]}',
 '{"P": [30]}',
 '{"P": [33, {"Z": [22, 29, 31]}]}',
 '{"P": [32]}

In [12]:
s = 'P[186, Z[191, 195, 193]]'

In [11]:
d = {
    "P": [
        1,
        {
            "Z": [
                2,
                3,
                4
            ]
        }
    ]
}

In [12]:
d

{'P': [1, {'Z': [2, 3, 4]}]}

In [18]:
import json

In [22]:
json.dumps(d)

'{"P": [1, {"Z": [2, 3, 4]}]}'

In [20]:
json.loads(str(d))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [9]:
def increase_int(d, increase):
    if isinstance(d, dict):
        for key, value in d.items():
            d[key] = increase_int(value, increase)
        return d
    elif isinstance(d, list):
        return [
            increase_int(elem, increase)
            for elem in d
        ]
    else:
        return d + increase

In [23]:
increase_int(d, 6)

{'P': [64, {'Z': [65, 66, 67]}]}

## Subset-test

In [1]:
from ptgnn.dataset import RSDataset, BaceDataset
ds = RSDataset(split='val')
# ds = BaceDataset(split='val')

In [2]:
from ptgnn.loading.subsetting import subset_dataset
sub_ds = subset_dataset(ds, 100)

In [3]:
sub_ds

RSDataset(100)